# Submitted by:
 
* Name 1: Bar Kinreich 313178345
* Name 2: Neta Dor 206759623

## To be submitted by: 20.2.2022

## Titanic survival prediction



In this part we will attempt to predict who survived on the Titanic. The data set we use has the following vairbales (features/attributes):


|  Variable   |          Definition          |              Key/Values                 |
|:-----------:|:----------------------------:|:---------------------------------------:|
| PassengerId | Index                        |integer                                  |
| Pclass      | Ticket class                 |1=1st, 2=2nd, 3=3rd                      |
| Name        | Name of passenger            |string                                   |
| Sex         | Sex                          |male, female                             |
| Age         | Age in years                 |integer                                  |
| SibSp       | # of siblings/spouses aboard |integer                                  |
| Parch       | # of parents/children aboard |integer                                  |
| Ticket      | Ticket number                |string                                   |
| Fare        | Ticket fare                  |float                                    |
| Cabin       | Cabin number                 |a code                                   |
| Embarked    | Port of Embarkation          |C=Cherbourg, Q=Queenstown, S=Southampton |
| **Survived**| Predicted varibale           |0=No, 1=Yes                              |


**Goal:**

This part of the project is a competetive one. The goal is to produce the best prediciton you can
<br><br>


**Methodogology**

So far you only know few methods for classification: KNN, logistic regression and SVM. You can use each one of them. You can also use linear regression, but then you need to convert the output to 0s and 1s (this is not a straight forward use of linear regression but a possible one). You may want to choose which features to use (it could be that some features are not useful). Further, some features have missing values. You will need to decide how to handle this (for instance: drop rows with missing values, place an avergae value in those rows, or some other method of your choosing). Another matter to consider is handling non-numeric values. For example, sex is non-numeric. You may choose to drop non-numeric features, or you could convert them to numeric values (if such conversion makes sense).
Also you will need to consider splitting the data into a training set and a test set so as to avoid tailoring the solution (overfitting) to the data you have.
<br><br>
Bottom line: use everything we talked about in class in order to learn the best model.


**Model Output**

Your model needs to produce a prediciton for each data sample (row), which is 0 (did not survive) or 1 (survived). 
<br><br>

**Scoring your model**

Your model performance will be assessed on test data that is not available to you. As mentioned above, this part of the project is a competition, where the goal is to achieve highest model accuracy

<br><br>
**Final Note**

This part of the project is open ended, in that you are not given small and specific tasks. However, you are already familiar with all the components needed to succeed. Specifically, reading data into pandas dataframe, dropping columns, dropping rows, changing value of features, splitting data into train and test subsets, performing model using sklearn library, and using cross validation. So don't panic...


GOOD LUCK

## Imporing the Libraries

In [ ]:
# linear algebra
import numpy as np 

# data processing
import pandas as pd 
from sklearn import preprocessing


# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import chi2, SelectKBest, f_classif
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn import svm
import re

## Getting the Data

In [ ]:
titanic_data = pd.read_csv("titanic_train.csv")

# Data Exploration
print("Info: \n")
print(titanic_data.info())
print("\n Describe: \n")
print(titanic_data.describe())

## Data Exploration

In [ ]:
# Missing values explotarion
total = titanic_data.isnull().sum().sort_values(ascending=False)
percent_1 = titanic_data.isnull().sum()/titanic_data.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

# The Embarked feature has only 2 missing values, which can easily be filled. It will be much more tricky, 
# to deal with the ‘Age’ feature, which has 177 missing values. 
#The ‘Cabin’ feature needs further investigation, but it looks like that we might want to drop it from the dataset, 
# since 77 % of it are missing.

In [ ]:
# Find means of each feature in order to compare between different genders
titanic_data.groupby('Sex').mean()

# We can see that males had a much lower chance of survival then women

In [ ]:
# Next, we will try to understand which features could contribute to a high survival rate by using plots

# Age and Sex
survived = 'survived'
not_survived = 'not survived'
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10, 4))
women = titanic_data[titanic_data['Sex']=='female']
men = titanic_data[titanic_data['Sex']=='male']
ax = sns.histplot(women[women['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[0], kde =False, color = 'skyblue')
ax = sns.histplot(women[women['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[0], kde =False, color = 'pink')
ax.legend()
ax.set_title('Female')
ax = sns.histplot(men[men['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[1], kde = False, color = 'skyblue')
ax = sns.histplot(men[men['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[1], kde = False, color = 'pink')
ax.legend()
_ = ax.set_title('Male')

# Since there seem to be certain ages, which have increased odds of survival and because I want every feature to be roughly on the same scale, I will create age groups later on.



In [ ]:
# Embarked, Pclass and Sex:
FacetGrid = sns.FacetGrid(titanic_data, row='Embarked', height=4.5, aspect=1.6)
FacetGrid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette=None,  order=None, hue_order=None )
FacetGrid.add_legend()

# We can see that women who embarked in port Q had a very high survival rate, while men who embarked in port C had a high survival rate. It seems that one gender's survival comes at the price of the other, as they have an inverse coorelation between them.
# This is except for both women and men that embarked in port S and were in class 3, where both genders had a low survival rate.

In [ ]:
# Pclass:
sns.barplot(x='Pclass', y='Survived', data=titanic_data)

# We can see that there is a very high correlation between Pclass and Survived, those in class 1 had a much higher survival rate then those in class 3

# Data Preprocessing

In [ ]:
# We would like to delete the columns that we believe don't contribute to the survival target
titanic_data = titanic_data.drop(['PassengerId'], axis=1)
titanic_data = titanic_data.drop(['Ticket'], axis=1)

### Missing Values

In [ ]:
# Cabin 
# A cabin number contains the letter refers to the deck. 
# Therefore we’re going to extract these and create a new feature, that contains a persons deck. 
# Afterwords we will convert the feature into a numeric variable. 
# The missing values will be converted to zero.

deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}

titanic_data['Cabin'] = titanic_data['Cabin'].fillna("U0")
titanic_data['Deck'] = titanic_data['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
titanic_data['Deck'] = titanic_data['Deck'].map(deck)
titanic_data['Deck'] = titanic_data['Deck'].fillna(0)
titanic_data['Deck'] = titanic_data['Deck'].astype(int)
# we can now drop the cabin feature
titanic_data = titanic_data.drop(['Cabin'], axis=1)

In [ ]:
# Age
# we create an array that contains random numbers, which are computed based on the mean age value in regards to the standard deviation and is_null.

mean = titanic_data["Age"].mean()
std = titanic_data["Age"].std()
is_null = titanic_data["Age"].isnull().sum()
# compute random numbers between the mean, std and is_null
rand_age = np.random.randint(mean - std, mean + std, size = is_null)
# fill NaN values in Age column with random values generated
age_slice = titanic_data["Age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
titanic_data["Age"] = age_slice
titanic_data["Age"] = titanic_data["Age"].astype(int)

# check that we got rid of the nulls
titanic_data["Age"].isnull().sum()


In [ ]:
# Embarked
# Embarked feature has only 2 missing values, we will  fill these with the most common one.

titanic_data['Embarked'].describe()
# because the top value is 'S', we will fill the missing values with it.
titanic_data['Embarked'] = titanic_data['Embarked'].fillna('S')

# check that we got rid of the nulls
titanic_data["Embarked"].isnull().sum()

### Converting Features

In [ ]:
# In order to see how to combine the "Title" coloumn and change it to categorial, we will first check what titles are in our dataset

titanic_data['Title'] = titanic_data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(titanic_data['Title'], titanic_data['Sex'])

In [ ]:
# Name 
# We created a new feature called Title from the Name feature


titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

# extract titles
titanic_data['Title'] = titanic_data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
# replace titles with a more common title or as Other
titanic_data['Title'] = titanic_data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                        'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
titanic_data['Title'] = titanic_data['Title'].replace('Mlle', 'Miss')
titanic_data['Title'] = titanic_data['Title'].replace('Ms', 'Miss')
titanic_data['Title'] = titanic_data['Title'].replace('Mme', 'Mrs')
# convert titles into numbers
titanic_data['Title'] = titanic_data['Title'].map(titles)
# filling NaN with 0, to get safe
titanic_data['Title'] = titanic_data['Title'].fillna(0)
titanic_data = titanic_data.drop(['Name'], axis=1)

# check that we got rid of the nulls
titanic_data["Title"].isnull().sum()

In [ ]:
# Sex
# Convert ‘Sex’ feature into numeric: male = 0, female = 1

genders = {"male": 0, "female": 1}

titanic_data['Sex'] = titanic_data['Sex'].map(genders)

In [ ]:
# Embarked
ports = {"S": 1, "C": 2, "Q": 3}

titanic_data['Embarked'] = titanic_data['Embarked'].map(ports)

In [ ]:
# SibSp and Parch
# combined feature, that shows the total number of relatives, a person has on the Titanic.
#  I will create it below and also a feature that shows if someone is not alone.

titanic_data['relatives'] = titanic_data['SibSp'] + titanic_data['Parch']
titanic_data.loc[titanic_data['relatives'] > 0, 'not_alone'] = 0
titanic_data.loc[titanic_data['relatives'] == 0, 'not_alone'] = 1
titanic_data['not_alone'] = titanic_data['not_alone'].astype(int)

titanic_data['relatives'].value_counts()

### Creating Categories 

In [ ]:
# Age
# According to the graph we did before, there seems to be certain age groups that have a higher chance for survival, so we will use them in order to create the age groups. 
# Also, we will want to make sure that the data is evenly distributed so that a large part of our data doesn't fall into one category
# We decided not to use qcut as the ages have meaning (infants vs. children vs. adults vs. senr)

titanic_data['Age'] = titanic_data['Age'].astype(int)
titanic_data.loc[ titanic_data['Age'] <= 11, 'Age'] = 1
titanic_data.loc[(titanic_data['Age'] > 11) & (titanic_data['Age'] <= 18), 'Age'] = 2
titanic_data.loc[(titanic_data['Age'] > 18) & (titanic_data['Age'] <= 24), 'Age'] = 3
titanic_data.loc[(titanic_data['Age'] > 24) & (titanic_data['Age'] <= 30), 'Age'] = 4
titanic_data.loc[(titanic_data['Age'] > 30) & (titanic_data['Age'] <= 37), 'Age'] = 5
titanic_data.loc[(titanic_data['Age'] > 37) & (titanic_data['Age'] <= 45), 'Age'] = 6
titanic_data.loc[ titanic_data['Age'] > 45, 'Age'] = 7

# let's see how it's distributed 
titanic_data['Age'].value_counts()


In [ ]:
# Fare
# we will use 'qcut' to help us decide the categories for the fare
pd.qcut(titanic_data["Fare"],6)

In [ ]:
titanic_data.loc[ titanic_data['Fare'] <= 7.775, 'Fare'] = 1
titanic_data.loc[(titanic_data['Fare'] > 7.775) & (titanic_data['Fare'] <= 8.458), 'Fare'] = 2
titanic_data.loc[(titanic_data['Fare'] > 8.458) & (titanic_data['Fare'] <= 14.2), 'Fare']   = 3
titanic_data.loc[(titanic_data['Fare'] > 14.2) & (titanic_data['Fare'] <= 26.25), 'Fare']   = 4
titanic_data.loc[(titanic_data['Fare'] > 26.25) & (titanic_data['Fare'] <= 53.1), 'Fare']   = 5
titanic_data.loc[ titanic_data['Fare'] > 53.1, 'Fare'] = 6
titanic_data['Fare'] = titanic_data['Fare'].astype(int)

In [ ]:
titanic_data['Fare'].value_counts()

In [ ]:
# Let's create a heatmap to understand what features has strong correlation, to make a new feature out of them
sns.heatmap(titanic_data.corr())

In [ ]:
# We saw that Sex and Title has strong correlation with the Survived target, therefore we decided to create a new feature which includes both of them
titanic_data['Sex_Title']= titanic_data['Sex'] + titanic_data['Title']

# Building a Model

In [ ]:
# Splitting into test and training data

training_data = titanic_data.sample(frac=0.8, random_state=25)
test_data = titanic_data.drop(training_data.index)

#Creating X table of features and Y table of the target feature:
X_train = training_data.drop("Survived", axis=1)
Y_train = training_data["Survived"]
X_val = test_data.drop("Survived", axis=1)
Y_val = test_data["Survived"]


# Normalization
# z-score scaling
zscore_scale = preprocessing.StandardScaler()
zscore_scale.fit(X_train)
Xtrain_zscore = zscore_scale.transform(X_train)
Xtest_zscore = zscore_scale.transform(X_val)

# min-max scaling
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
minmax_scale.fit(X_train)
Xtrain_minmax = minmax_scale.transform(X_train)
Xtest_minmax = minmax_scale.transform(X_val)

In [ ]:
# After numerous tests we found that the min-max feature has a better accuracy then Z-score

X_train = Xtrain_minmax
X_val = Xtest_minmax

### Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_val)

acc_log = round(logreg.score(X_train, Y_train) * 100, 2)

### KNN

In [ ]:
# KNN 
knn = KNeighborsClassifier(n_neighbors = 7) 
knn.fit(X_train, Y_train)  
Y_pred = knn.predict(X_val)  
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)

# with for loop we found that k=3 has the highest accuracy, but with KNN model we need to see how it works after doing K-folds to see that the accuracy is percised.
# after testing with k-folds we saw that 7 brings the best scores

print(classification_report(Y_val, Y_pred))

acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

### Support Vector Machine

In [ ]:
linear_svc = SVC()

# Optimization of the hyper-parameters of the model
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

linear_svc1 = GridSearchCV(linear_svc, param_grid, refit = True, verbose = 3)

linear_svc1.fit(X_train, Y_train)



In [ ]:
# print best parameter after tuning

linear_svc1.best_params_

# print how our model looks after hyper-parameter tuning
print(linear_svc1.best_estimator_)

linear_svc = SVC(C=1, gamma=1,kernel="rbf")
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_val)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
# print classification report
print(classification_report(Y_val,Y_pred))

In [ ]:
results = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression'],
    'Score': [acc_linear_svc, acc_knn, acc_log]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df

## Conclusion: 
### We can see that the model with the best performance by accuracy is the KNN model, with the highest accuracy score

# Test Data Processing

In [ ]:
def data_preprocessing(test_data):
    titanic_test = pd.read_csv(test_data)
    titanic_test = titanic_test.drop(['Ticket'], axis=1)
    
    #Missing values
    # Deck
    deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
    titanic_test['Cabin'] = titanic_test['Cabin'].fillna("U0")
    titanic_test['Deck'] = titanic_test['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    titanic_test['Deck'] = titanic_test['Deck'].map(deck)
    titanic_test['Deck'] = titanic_test['Deck'].fillna(0)
    titanic_test['Deck'] = titanic_test['Deck'].astype(int)
    # We can now drop the cabin feature
    titanic_test = titanic_test.drop(['Cabin'], axis=1)
    
    # Age
    mean = titanic_test["Age"].mean()
    std = titanic_test["Age"].std()
    is_null = titanic_test["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = titanic_test["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    titanic_test["Age"] = age_slice
    titanic_test["Age"] = titanic_test["Age"].astype(int)
    
    #Embarked
    titanic_test['Embarked'] = titanic_test['Embarked'].fillna('S')
    
    # Name 
    titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    # extract titles
    titanic_test['Title'] = titanic_test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Other
    titanic_test['Title'] = titanic_test['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
    titanic_test['Title'] = titanic_test['Title'].replace('Mlle', 'Miss')
    titanic_test['Title'] = titanic_test['Title'].replace('Ms', 'Miss')
    titanic_test['Title'] = titanic_test['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    titanic_test['Title'] = titanic_test['Title'].map(titles)
    # filling NaN with 0
    titanic_test['Title'] = titanic_test['Title'].fillna(0)
    titanic_test = titanic_test.drop(['Name'], axis=1)
    
    # Sex
    # Convert ‘Sex’ feature into numeric: male = 0, female = 1

    genders = {"male": 0, "female": 1}

    titanic_test['Sex'] = titanic_test['Sex'].map(genders)
    
    # Embarked
    ports = {"S": 1, "C": 2, "Q": 3}

    titanic_test['Embarked'] = titanic_test['Embarked'].map(ports)
    
    # SibSp and Parch

    titanic_test['relatives'] = titanic_test['SibSp'] + titanic_test['Parch']
    titanic_test.loc[titanic_test['relatives'] > 0, 'not_alone'] = 0
    titanic_test.loc[titanic_test['relatives'] == 0, 'not_alone'] = 1
    titanic_test['not_alone'] = titanic_test['not_alone'].astype(int)
    
    # Age
    
    titanic_test['Age'] = titanic_test['Age'].astype(int)
    titanic_test.loc[ titanic_test['Age'] <= 11, 'Age'] = 1
    titanic_test.loc[(titanic_test['Age'] > 11) & (titanic_test['Age'] <= 18), 'Age'] = 2
    titanic_test.loc[(titanic_test['Age'] > 18) & (titanic_test['Age'] <= 24), 'Age'] = 3
    titanic_test.loc[(titanic_test['Age'] > 24) & (titanic_test['Age'] <= 30), 'Age'] = 4
    titanic_test.loc[(titanic_test['Age'] > 30) & (titanic_test['Age'] <= 37), 'Age'] = 5
    titanic_test.loc[(titanic_test['Age'] > 37) & (titanic_test['Age'] <= 45), 'Age'] = 6
    titanic_test.loc[ titanic_test['Age'] > 45, 'Age'] = 7
    
    # Fare
    
    titanic_test.loc[ titanic_test['Fare'] <= 7.775, 'Fare'] = 1
    titanic_test.loc[(titanic_test['Fare'] > 7.775) & (titanic_test['Fare'] <= 8.458), 'Fare'] = 2
    titanic_test.loc[(titanic_test['Fare'] > 8.458) & (titanic_test['Fare'] <= 14.2), 'Fare']   = 3
    titanic_test.loc[(titanic_test['Fare'] > 14.2) & (titanic_test['Fare'] <= 26.25), 'Fare']   = 4
    titanic_test.loc[(titanic_test['Fare'] > 26.25) & (titanic_test['Fare'] <= 53.1), 'Fare']   = 5
    titanic_test.loc[ titanic_test['Fare'] > 53.1, 'Fare'] = 6
    titanic_test['Fare'] = titanic_test['Fare'].astype(int)
    
    # Additional coloumns
    
    titanic_test['Sex_Title']= titanic_test['Title'] + titanic_test['Sex']
    
    return titanic_test 
    

### Enter your test data here under test data

In [ ]:
titanic_test = data_preprocessing("titanic_test")

In [ ]:
# Because we don't know if the test dataset will include the target variable, we will fit the dataset for both scenarios

X_train = titanic_data.drop("Survived", axis=1)
Y_train = titanic_data["Survived"]
if "Survived" in titanic_test.columns:
    Y_test = titanic_test["Survived"]
    X_test = titanic_test.drop(["PassengerId", "Survived"], axis = 1).copy()
else:
    X_test = titanic_test.drop("PassengerId", axis=1).copy()

In [ ]:
# min-max scaling to the test data
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
minmax_scale.fit(X_train)
X_train = minmax_scale.transform(X_train)
X_test = minmax_scale.transform(X_test)

In [ ]:
knn.fit(X_train,Y_train)
Y_pred = knn.predict(X_test)  
acc_knn = round(knn.score(X_test, Y_test) * 100, 2)
print(acc_knn)